Import the packages

In [1]:
import numpy as np
import vector

Define masses of the parent and daughter particles (in $MeV/c^2$)

In [3]:
mp = 105.7 #muon
m1 = 0.511 #electron
m2 = 0.12*1e-6 #electron neutrino is less than this
m3 = 0.12*1e-6 #muon neutrino is less than this

Sample $s_{12}$

In [4]:
#number of simulations
N = 1000

s12max = (mp-m3)**2
s12min = (m1+m2)**2
s12 = np.random.uniform(s12min,s12max,N)

Calculate magnitude of momentum of daughter particles in COM frame when the parent particle decays into $p_{12}$ and $p_3$

In [5]:
p3 = np.sqrt(s12**2+(m3**2-mp**2)**2-(2*s12*(m3**2+mp**2)))/(2*mp)


Sample Angles in the lab rest frame

In [6]:
cos_theta = np.random.uniform(-1,1,N)
phi = np.random.uniform(0,2*np.pi,N)
theta = np.arccos(cos_theta)

Spatial momentum components and energy of daughter particle $p_3$

In [7]:
p3x = p3*np.sin(theta)*np.cos(phi)
p3y = p3*np.sin(theta)*np.sin(phi)
p3z = p3*cos_theta

E3 = np.sqrt(m3**2+p3**2)

4-vector of daughter particle $p_3$

In [8]:
daughter3 = vector.array(
    {
        "E": np.full(N, E3),
        "px": p3x,
        "py": p3y,
        "pz": p3z,
    }
)

Now we look at the decay of $p_{12}$ to $p_1$ and $p_2$

In [9]:
p12 = np.sqrt(m1**4+(m2**2-s12)**2-(2*(m1**2)*(m2**2+s12)))/(2*np.sqrt(s12))

cos_theta12 = np.random.uniform(-1,1,N)
phi12 = np.random.uniform(0,2*np.pi,N)
theta12 = np.arccos(cos_theta)

p12x = p12*np.sin(theta12)*np.cos(phi12)
p12y = p12*np.sin(theta12)*np.sin(phi12)
p12z = p12*cos_theta12

E1 = np.sqrt(m1**2+p12**2)
E2 = np.sqrt(m2**2+p12**2)

daughter1 = vector.array(
    {
        "E": np.full(N, E1),
        "px": p12x,
        "py": p12y,
        "pz": p12z,
    }
)

daughter2 = vector.array(
    {
        "E": np.full(N, E2),
        "px": -p12x,
        "py": -p12y,
        "pz": -p12z,
    }
)

This is all in the rest frame of $p_{12}$ so we Lorentz boost this to the rest frame of the parent particle.

Using conservation of energy, $E_{init} = m_p$ and $E_{f}=E_{12}+E_3$. Therefore $E_{12}=m_p-E_3$ and by conservation of spatial momentum, it has the negative signs of $p_3$'s spatial momentum.

In [11]:
daughter12 = vector.array({
    "E": mp - daughter3.E,
    "px": -daughter3.px,
    "py": -daughter3.py,
    "pz": -daughter3.pz,
})

boosted_1 = daughter1.boostCM_of_p4(daughter12)
boosted_2 = daughter2.boostCM_of_p4(daughter12)

print("Daughter 1:", boosted_1[0:3])
print("Daughter 2:", boosted_2[0:3])
print("Daughter 3:", daughter3[0:3])


Daughter 1: [( 7.66107036,  14.52031772,   0.55055921, 15.69760105)
 ( 1.88088982, -49.59508079, -43.70398201, 51.12650421)
 (-7.7752632 , -23.18693367,  22.10082496, 39.9750846 )]
Daughter 2: [(35.26400771,  8.28206973, -15.57365151, 39.12799965)
 (-7.78261922, 46.91230458,  42.41109602, 47.96295479)
 (14.9803694 , 36.09397432,  -9.13756728, 46.0640074 )]
Daughter 3: [(42.92507808, 22.80238745, -15.0230923 , 50.8743993)
 (-5.9017294 , -2.68277621,  -1.29288599,  6.610541 )
 ( 7.2051062 , 12.90704065,  12.96325768, 19.660908 )]
